In [2]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
numpy.set_printoptions(threshold=sys.maxsize)

4.1.1


# Extract marks from images to dictionary and save

In [1]:
path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/"

def get_marks(filepath):
    marks = []
    img = cv2.imread(filepath)
    validation_sum = 3*255 - 50
    for x,y in np.ndindex(img.shape[:2]):
        if np.sum(img[(x,y)]) > validation_sum:
            marks.append((y,x))
    print(marks)
    return marks

def get_all_marks():
    marks = dict()
    filenames = [f for f in sorted(listdir(path)) if isfile(join(path, f))]
    for i in tqdm(range(len(filenames))):
        filename = filenames[i]
        print(path, filename)
        marks[filename] = get_marks(path + filename)
    return marks

marks = get_all_marks()
np.save(path+"eval/test_marks", marks)


NameError: name 'listdir' is not defined

# Evaluate

In [3]:
def dist(a,b):
    return np.linalg.norm(np.array(a)-np.array(b))
    
path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/eval/"
pred = np.load(path+"pred_centers.npy", allow_pickle=True).item()
test = np.load(path+"test_marks.npy", allow_pickle=True).item()
filenames = test.keys()
deviations = []
missed_predictions = 0
well_predictions = 0
redundant_predictions = 0
acceptance_thr = 15
for filename in filenames:
    marks = test[filename]
    center = pred[filename][0]
    distances = [dist(center, x) for x in marks]
    if distances:
        print(distances)
        min_distance = int(np.min(distances))
        deviations.append(min_distance)
        if min_distance <= acceptance_thr:
            well_predictions += 1
        else:
            missed_predictions += 1
    else:
        redundant_predictions += 1
        
    
error = np.mean(deviations)
print(deviations, error, " well:",well_predictions, " missed:", missed_predictions, " redundant:", redundant_predictions)

[5.0990195135927845, 186.02419197512995]
[188.00066489244128, 0.5]
[6.082762530298219, 187.0240626229684]
[187.01671048331482, 0.5]
[190.04210059878838, 3.605551275463989]
[201.02238681301145, 1.4142135623730951]
[6.519202405202649, 225.5271602268782]
[1.118033988749895]
[2.5]
[2.0]
[5.70087712549569]
[3.5]
[188.00066489244128, 4.924428900898052]
[7.433034373659253]
[123.4878536537096]
[5.385164807134504, 187.04277585621958]
[88.31902399823042]
[4.527692569068709]
[4.716990566028302]
[237.5005263152063, 2.9154759474226504]
[207.5006024087641, 4.527692569068709]
[2.8284271247461903, 196.04081207748553]
[401.5526117459579, 195.5057543910153, 5.522680508593631]
[5.5901699437494745, 207.00543471126548, 408.06892800114053]
[216.23135757794242, 5.0990195135927845, 203.08865059377396]
[226.6991398307457, 2.9154759474226504]
[186.03292719301064, 2.692582403567252]
[250.28783430282823, 3.1622776601683795]
[3.2015621187164243]
[4.301162633521313]
[2.692582403567252]
[4.716990566028302]
[160.2817